In [ ]:
from ResearchMain import *

In [ ]:
DATA_PATH = os.getcwd() + '\\data\\EEMs\\'

In [ ]:
def from_file_path_get_name(path):
    """
    根据文件路径分割出文件名，并进行修饰
    :param path: 文件路径
    :return: 文件名
    """
    name = path.split('\\')[-1].split('.')[0]
    if name.startswith('1'):
        return 'D-' + name[1:]
    elif name.startswith('2'):
        return 'L-' + name[1:]
    elif name.startswith('3'):
        return 'W-' + name[1:]


def get_skip_rows(path):
    """
    读取txt文件，并在文件中查找含有'Data Points'的行，数据矩阵就在这一行的下面
    :param path: 文件路径
    :return: 数据矩阵开始的行号
    """
    f = open(path)
    for index, line in enumerate(f.readlines()):
        if 'Data Points' in line:
            return index + 1


In [ ]:
# 创建一个空df，用于保存所有df
col_name = ['ID', 'SkipRows', 'Path']
eems_df = pd.DataFrame(columns=col_name)


def get_all_files(root_path=DATA_PATH, suffix='.TXT'):
    """
        递归获取文件夹内所有文件
    :param root_path: 文件夹路径
    :param suffix: 文件后缀
    """
    # 获取的当前文件夹路径不是当前文件所在的文件夹路径，而是Python程序运行时CMD窗口所运行的文件夹！
    global eems_df
    file_list = os.listdir(root_path)
    # 判断路径为文件还是文件夹
    if file_list:
        for f in file_list:
            temp_path = root_path + f
            if os.path.isdir(temp_path):  # 是文件夹，进行递归操作
                get_all_files(temp_path)
            elif os.path.isfile(temp_path):  # 是文件
                if f.endswith(suffix):  # 是数据文件
                    name = from_file_path_get_name(temp_path)
                    skip_rows = get_skip_rows(temp_path)
                    _ = pd.DataFrame([[name, skip_rows, temp_path]], columns=col_name)
                    eems_df = pd.concat([eems_df, _])
            else:
                # 特殊无后缀文件
                print("it's a special file(socket,FIFO,device file):" + temp_path)
    else:
        hues.warn(f'The path <{root_path}> don\'t have files.')


get_all_files()
eems_df

In [ ]:
# 预处理
eems_df = eems_df.set_index('ID')  # 设置ID列为索引

# 向df中新增季节、河流两列，用于数据区分与排序，整体上的顺序是：南淝河<派河<杭埠河, 枯水期<平水期<丰水期
eems_df['Period'] = eems_df.apply(lambda x: 'Dry' if 'D' in x.name else ('Wet' if 'W' in x.name else 'Level'),
                                  axis=1) + ' Season'


def check_river(s):
    """
    根据每一行的ID判断每一行数据属于哪一行
    :param s:
    :return:
    """

    if 'BlankSample' in s.name:
        return np.nan
    else:
        return_str = ''
        if 'N' in s.name:
            return_str = 'Nanfei'
        elif 'P' in s.name:
            return_str = 'Pai'
        elif 'H' in s.name:
            return_str = 'Hangbu'
        return_str += ' River'
        return return_str


eems_df['River'] = eems_df.apply(check_river, axis=1)
eems_df['River'] = eems_df['River'].astype(river_order)
eems_df['Period'] = eems_df['Period'].astype(period_order)
eems_df = eems_df.sort_values(by=['Period', 'River'])
eems_df

In [ ]:
def handle_eems_df(df_id, info_df=eems_df):
    """
    根据输入的df_id去eems_df表中查找df，然后对df进行数据处理，最后返回。样本数据处理：1、去除拉曼散射（减去空白样）；2、过滤负值；3、消除瑞利散射
    :param info_df: 用于查找信息的表
    :param df_id: df的ID，根据eems_df表中的ID为准
    :return: 返回修改后的df
    """
    if 'BlankSample' in df_id:  # 过滤出空白样
        return None
    else:
        try:
            search_info = info_df.loc[df_id]
        except NameError:
            hues.error(f'未在"info_df"中查询到ID={df_id}的数据行信息！！！')
            return None
        else:
            # 读取数据表
            df = pd.read_table(search_info['Path'], skiprows=search_info['SkipRows'], index_col=0)
        # 1、减去空白样（拉曼散射）
        blank_search_info = None
        if df_id.startswith('D-'):  # 枯水期
            blank_search_info = info_df.loc['D-BlankSample']
        elif df_id.startswith('L-'):  # 平水期
            blank_search_info = info_df.loc['L-BlankSample']
        elif df_id.startswith('W-'):  # 丰水期
            blank_search_info = info_df.loc['W-BlankSample']
        df_blank = pd.read_table(blank_search_info['Path'], skiprows=blank_search_info['SkipRows'], index_col=0)
        modify_df = df - df_blank
        # 2、扣除空白样品后，需要将df中的所有负值清除（Tips：负值在原理上没有意义，但是在实际实验检测过程中出现是很正常的）
        modify_df = modify_df.applymap(lambda x: 0 if x < 0 else x)
        # 3、消除瑞利散射
        for i in range(modify_df.shape[0]):  # 遍历ex
            for j in range(modify_df.shape[1]):  # 遍历em
                ex = 200 + i * 5  # ex范围为200~450，间隔为5
                em = 280 + j * 5  # em范围为280~550，间隔为5
                if ex + 455 > em > ex + 145:
                    modify_df.iloc[i, j] = 0  # 不能使用0, 因为0是有意义的数据
        # 4、将columns转化为数字类型，方便后续使用
        modify_df.columns = modify_df.columns.astype(float)

        return modify_df

In [ ]:
handle_eems_df('D-H1')

In [ ]:
# 绘图全局设置——设置字体(主要字体/数学公式字符集)
rc = {'font.family': 'Times New Roman',  # 默认字体
      'mathtext.fontset': 'stix',  # 数学字符集
      'font.size': 60
      }
mpl.rcParams.update(rc)  # 根据dict一次性更新很多参数
# 绘图分区
subarea = {
    'Ⅰ': [(200, 280), (250, 330)],  #
    'Ⅱ': [(200, 330), (250, 380)],
    'Ⅲ': [(200, 380), (250, 550)],
    'Ⅳ': [(250, 280), (450, 330)],
    'Ⅴ': [(250, 330), (450, 550)]
}

plot_bottom_distance = {
    'D': 0.2,
    'L': 0.15,
    'W': 0.15
}

In [ ]:
# 绘图可视化
PERIOD = 'W'  # PERIOD=D, L, W
plot_df = eems_df[eems_df['Period'].str.startswith(PERIOD) & ~pd.isnull(eems_df['River'])].copy()
PIG_NUM = plot_df.shape[0]  # 子图总个数
COL_NUM = 6  # 图片中子图的列数
ROW_NUM = math.ceil(PIG_NUM // COL_NUM)

# 根据每条河的ID重新排序
plot_df['SortNum'] = plot_df.index.str[3:].astype(float)
plot_df = plot_df.sort_values(by=['River', 'SortNum'])

In [ ]:
# 布图设置
fig, ax_arr = plt.subplots(nrows=ROW_NUM, ncols=COL_NUM, figsize=(COL_NUM * 10, ROW_NUM * 10 + 3), sharex='all',
                           sharey='all')
fig.subplots_adjust(hspace=0.01, wspace=0.01)

# 默认绘图坐标轴后面是透明的，调整下成白色背景
fig.patch.set_alpha(1.0)
fig.patch.set_facecolor('white')

# 循环绘制三维荧光图
k = 0  # 循环控制变量，用于控制编号
for file_info in plot_df.iterrows():
    # 数据准备
    df_name = file_info[0]
    cur_df = handle_eems_df(df_name)  # 传入文件名，即可获得处理后的df
    X, Y, Z = np.array(cur_df.columns.astype('float')), np.array(cur_df.index), np.array(cur_df)
    Z_MIN, Z_MAX, Z_STEP = 0, np.max(Z), 10
    X, Y = np.meshgrid(X, Y)
    N = np.arange(Z_MIN, Z_MAX, Z_STEP)
    # 计算行、列编号
    row_num = k // COL_NUM
    col_num = k % COL_NUM
    cur_ax = ax_arr[row_num][col_num]
    # 绘图
    cur_plot = cur_ax.contourf(X, Y, Z, N, cmap=mpl.cm.jet)
    # 子图添加编号
    cur_ax.text(
        405,
        355,
        f'({df_name})',
        fontdict={'size': '50', 'color': 'white'},
        ha='center', va='center'
    )
    # 每行第一个子图设置y轴名称
    if k % COL_NUM == 0:
        cur_ax.set_ylabel('$Em(nm)$')
    # 最后一行子图设置x轴名称与x轴刻度
    if PIG_NUM - k <= COL_NUM:
        plt.xticks(np.linspace(250, 400, 3, endpoint=True))
    cur_ax.set_xlabel('$Ex(nm)$')
    # 等高线
    # cur_ax.contour(X, Y, Z, Z_STEP)
    # 绘制区域分割线
    cur_ax.axvline(250, color='white', linestyle='--', linewidth=5)
    cur_ax.axhline(330, color='white', linestyle='--', linewidth=5)
    cur_ax.axhline(380, xmin=0, xmax=0.2, color='white', linestyle='--', linewidth=5)
    # 添加分区编号文字
    for area_name, coordinate in subarea.items():
        small_x, small_y, big_x, big_y = coordinate[0][0], coordinate[0][1], coordinate[1][0], coordinate[1][1]
        center_x = (big_x - small_x) / 2 + small_x
        center_y = (big_y - small_y) / 2 + small_y
        cur_ax.text(center_x, center_y, area_name, horizontalalignment='center', verticalalignment='center',
                    fontdict={'size': '40', 'color': 'white'}, )
        # 每次循环结束k+1
    k += 1

# 添加颜色条
fig.subplots_adjust(bottom=plot_bottom_distance[PERIOD])
cbar_ax = fig.add_axes([0.15, 0.1, 0.7, 0.01])  # [left, bottom, width, height]  # 控制颜色条位置、大小
fig.colorbar(cur_plot, cax=cbar_ax, label='Signal value', orientation='horizontal')  # cur_plot是for循环里面的变量，为啥可以在外部调用？？？

# 保存
plt.savefig(EXPORT_PATH + f'[{PERIOD}]底泥DOM三维荧光图', dpi=DPI, bbox_inches='tight')
# 清除matplotlib设置